# 7. Training & Inference 1 - Loss, Optimizer, Metric

> 데이터셋도 준비했고, 모델도 준비했으니 드디어 본격적으로 모델 학습 과정에 들어갑니다.

> 그 전에, 모델 학습에 필요한 요소(Loss, Optimizer, Metric)의 의미를 다뤄보겠습니다.

> 앞서 설명한 파이토치 프레임워크에서 각각의 요소가 표현될 수 있을지 알아보겠습니다.

> 그리고, Optimizer가 Learning rate를 스케쥴링 하는 방식에 대해서 다루겠습니다.(Optimizer의 종류에 대해서는 따로 설명하지 않습니다.)

> 마지막으로, 태스크의 속성과 데이터 분포에 따른 Metric의 중요성에 대해서 다루겠습니다.

<br>

## 7.1 Overview

- 본격적으로 모델 학습을 진행해보자.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1nCL1bu_qJq0SIIoXMmIaGPUgrN0BgJTA' width=800/>

<br>

- 학습 프로세스에 필요한 요소를 크게 아래와 같이 나눌 수 있습니다.
  - Loss
  - Optimizer
  - Metric

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=12SVGNz21HalR4p6wpCZjhcAeAA1_VEMR' width=800/>

<br>

## 7.2 Loss

- (오차) 역전파
- Loss 도 사실은 `nn.Module` Family
- `loss.backward()`
- Example: 조금 특별한 loss

<br>

### 7.2.1 복습: (오차) 역전파

- Error Backpropagation
- Loss 함수 = Cost 함수 = Error 함수

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1kE9fgXnB8FwIOzglGmtBOTAxwdvQVNM2' width=700/>

<br>

### 7.2.2 Loss 도 사실은 `nn.Module` Family

- `nn` 패키지에서 찾을 수 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1Xtk4x2S6NgI-rURg2BBnnWH2uQaDnAuI' width=900/>

<br>

### 7.2.3 `loss.backward()`

- 이 함수가 실행되면 모델의 파라미터의 grad 값이 업데이트 됩니다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1UH4d_v_70rPJR4dA1ccXCRfobYzdTx28' width=900/>

<br>

### 7.2.4 Exaxmple: 조금 특별한 loss

- 쉽게 말해서 Error를 만들어내는 과정에 양념을 치는 것

<br>

#### 7.2.4.1 Focal Loss

- Class Imbalance 문제가 있는 경우, 맞춘 확률이 높은 Class는 조금의 loss를, 맞춘 확률이 낮은 Class 는 Loss 를 훨씬 높게 부여

<br>

#### 7.2.4.2 Label Smoothing Loss

- 예측한 class 가 100% 그 클래스가 맞을까?
- Class target label 을 one-hot 표현으로 사용하기 보다는 조금 soft 하게 표현해서 일반화 성능을 높이기 위함
  - `[0, 1, 0, 0, ...]` -> `[0.025, 0.9, 0.025, 0.025, ...]`

<br>

## 7.3 Optimizer

- 어느 방향으로, 얼마나 움직일 지?
- LR Scheduler

<br>

### 7.3.1 어느 방향으로, 얼마나 움직일 지?

- 영리하게 움직일수록 수렴은 빨라진다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1HLXtzOnM5kBFPublWEBTBktiGAq_f7ZV' width=400/>

<br>

### 7.3.2 LR Scheduler

- 학습 시에 Learning rate 를 동적으로 조절할 수는 없을까?

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=17x-csSaH29zaZKaqN3k3evTLZsyMUIdP' width=800/>

<br>

#### 7.3.2.1 `StepLR`

- 특정 Step 마다 LR 감소
```python
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)
```

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=19pEzCQ0R9whLyeAsrwu3QiCFKMwBeu93' width=500/>

- 출처: https://www.kaggle.com/isbhargav/guide-to-pytorch-learning-rate-scheduling/log

<br>

#### 7.3.2.2 `CosineAnnealingLR`

- Cosine 함수 형태처럼 LR 을 급격히 변경
- 변화를 다양하게 줄 수 있는 장점이 있다.
- local minimum 을 효과적으로 탈출할 수 있다.
```python
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)
```

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1IPbyz4yvXNNAB5LEko9rmPdNB99vwSJY' width=500/>

- 출처: https://www.kaggle.com/isbhargav/guide-to-pytorch-learning-rate-scheduling/log

<br>

#### 7.3.2.3 `ReduceLROnPlateau`

- 더 이상 성능 향상이 없을 때 LR 감소
- 일반적으로 가장 많이 사용
```python
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
...
scheduler.step(metric)
```

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1vQYaDbNsIPTxKerDz8rqNYMp0mkEXFXp' width=500/>

- 출처: https://www.kaggle.com/isbhargav/guide-to-pytorch-learning-rate-scheduling/log

<br>

## 7.4 Metric

- 모델의 평가
- Score 의 허와 실

<br>

### 7.4.1 모델의 평가

- Classification
  - Accuracy
  - F1-score
  - precision
  - recall
  - ROC & AUC
- Regression
  - MAE
  - MSE
- Ranking
  - 추천 시스템
  - 순서가 중요할 때
  - MRR
  - MDCG
  - MAP

- 학습에 직접적으로 사용되는 것은 아니지만 학습된 모델을 객관적으로 평가할 수 있는 지표가 필요

<br>

### 7.4.2 Metric 의 허와 실

- 제가 만든 모델은 정확도가 90% 입니다. ^^
- 오른쪽 그림에서 실제값이 1일 때 제대로 맞춘 것은 50% 밖에 안된다. (왼쪽은 70%)
- 클래스의 분포가 imbalance 할 때는 accuracy 는 잘못된 말을 할 수 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1GHVbx10Hrjbl1K6qnRlWFoNwG0pCvI1w' width=800/>

<br>

### 7.4.3 올바른 Metric 의 선택

- 데이터 상태에 따라 적절한 Metric 을 선택하는 것이 필요
  - Class 별로 밸런스가 적절히 분포  
  $\rightarrow$ Accuracy
  - Class 별 밸런스가 좋지 않아서 각 클래스 별로 성능을 잘 낼 수 있는 지 확인 필요  
  $\rightarrow$ F1-score